In [1]:
from util import *

with open('raw/tracks_uri_2_id.json') as f:
    tracks_uri_2_id=json.load(f)
with open('raw/tracks_id_2_uri.json') as f:
    tracks_id_2_uri=json.load(f)


flag='small'
playlists_tra,playlists_tst,playlist_tracks_tra,playlist_tracks_tst,playlist_tracks_tru,tracks,playlists,playlist_tracks = load_small_data()
y_true=ytrue(playlists_tst,playlist_tracks_tru)

ptracks=pd.concat([playlist_tracks_tra,playlist_tracks_tst])
playlists=sorted(ptracks.pid.unique())
pid_2_ix={p:i for i,p in enumerate(playlists)}
track_ids=sorted(ptracks.track_id.unique())
tid_2_ix={t:i for i,t in enumerate(track_ids)}

r=np.zeros((ptracks.shape[0],))
c=np.zeros((ptracks.shape[0],))
d=np.ones((ptracks.shape[0],))
for i in tnrange(ptracks.shape[0]):
    r[i]=pid_2_ix[ptracks.iat[i,0]]
    c[i]=tid_2_ix[ptracks.iat[i,2]]
plays=csr_matrix((d,(r,c)),shape=(len(playlists),len(track_ids)))
plays

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, max=670263), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


<9990x171845 sparse matrix of type '<class 'numpy.float64'>'
	with 661198 stored elements in Compressed Sparse Row format>

In [19]:

class Als:
    
    def __init__(self,alpha=0.01,epsilon=100,factors=5,max_iter=100):
        self.rc=500
        
        self.alpha=alpha
        self.epsilon=epsilon
        self.factors=factors
        self.max_iter=max_iter
        
              
    def fit(self,r_all):
        
        r=r_all
        
        self.f_users=np.random.rand(r_all.shape[0],self.factors)
        self.f_items=np.random.rand(r_all.shape[1],self.factors)

        last_cost = np.inf    
        for it in tnrange(self.max_iter):
            a = np.dot(self.f_items.T,self.f_items)+self.alpha*np.eye(self.factors)
            b = np.dot(csr_matrix(self.f_items.T),r.T)
            self.f_users=np.dot(np.linalg.inv(a), b.toarray()).T
            
            a = np.dot(self.f_users.T,self.f_users)+self.alpha*np.eye(self.factors)
            b = np.dot(csr_matrix(self.f_users.T),r)
            self.f_items=np.dot(np.linalg.inv(a), b.toarray()).T
            
#             u = np.sum(np.sum(self.f_users**2))
#             m = np.sum(np.sum(self.f_items**2))
#             cost = self.alpha*(u + m)
            
#             for i in range(self.f_users.shape[0]):
#                 r_pred=np.dot(self.f_users[i],self.f_items.T)
#                 cost += np.sum(np.sum(r[i].toarray()-r_pred)**2)
                    
#             print('iter %d, cost %.2f'%(it+1,cost))

#             if np.isnan(cost) or (last_cost > cost and last_cost-cost < self.epsilon) or last_cost<cost:
#                 break

#             last_cost = cost
                
              
        return self

    def predict(self,y_tst,items,uid_2_ix):
        y_pred={}

        users=sorted(y_tst.pid.unique())
        for i in tnrange(len(users)):
            u = users[i]
            f_user=self.f_users[uid_2_ix[u]]
            r_items=np.dot(f_user,self.f_items.T)
            i_pred=np.argsort(r_items)
            
            seed_items=set(y_tst[y_tst.pid==u].track_id)
            pred_items=np.zeros((self.rc,))
            k=0
            for j in range(len(i_pred)-1,0,-1):
                itm=items[i_pred[j]] 
                if itm not in seed_items:
                    pred_items[k] = itm
                    k+=1
                    if k == self.rc:
                        break
            assert k==self.rc
            
            y_pred[u]=pred_items
                    
        return y_pred
                
    def get_params(self,deep=True):
        return {'epsilon':self.epsilon,
                'factors':self.factors,'max_iter':self.max_iter}
    
    def set_params(self,**params):
        self.epsilon=params['epsilon']
        self.factors=params['factors']
        self.max_iter=params['max_iter']
        

als=Als(alpha=1e-1,factors=100,max_iter=1,epsilon=1e2)
als.fit(plays)
y_prd1=als.predict(playlist_tracks_tst,track_ids,pid_2_ix)


HBox(children=(IntProgress(value=0, max=1), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


HBox(children=(IntProgress(value=0, max=90), HTML(value='')))

In [5]:
import title

playlists_tra['title']=playlists_tra.name.apply(title.build_title)
playlists_tst['title']=playlists_tst.name.apply(title.build_title)
playlists_tra.index=playlists_tra.pid
playlist_tracks_tra.index=playlist_tracks_tra.pid

p_no_title=set(playlists_tst.pid)-set(playlist_tracks_tst.pid)
y_prd2=title.recommend(playlists_tra,playlists_tst[playlists_tst.pid.apply(lambda p:p in p_no_title)],
                         playlist_tracks_tra,playlist_tracks_tst,tracks)
y_pred=y_prd1.copy()
print('r_prec %.3f, ndcg %.3f, click %.3f'%(r_prec(y_true,y_pred),ndcg(y_true,y_pred),clicks(y_true,y_pred)))
y_pred.update(y_prd2)
assert y_pred.keys()==y_true.keys()
print('r_prec %.3f, ndcg %.3f, click %.3f'%(r_prec(y_true,y_pred),ndcg(y_true,y_pred),clicks(y_true,y_pred)))

HBox(children=(IntProgress(value=0, max=10), HTML(value='')))

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


r_prec 0.337, ndcg 0.431, click 12.089
r_prec 0.318, ndcg 0.411, click 13.630


In [15]:
# %%time
a=np.random.rand(1009000,10)
b=np.random.rand(10,2262292)
c=np.dot(a,b)
print(c.shape)

MemoryError: 

/home/cq/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [ ]:

# class WAls(Als):
    
#     def __init__(self,alpha=0.01,epsilon=100,factors=5,max_iter=100,beta=1):
#         super().__init__(alpha,epsilon,factors,max_iter)
#         self.beta=beta
              
#     def fit(self,r,w):
        
#         self.f_users=np.random.rand(r.shape[0],self.factors)
#         self.f_items=np.random.rand(r.shape[1],self.factors)

#         last_cost = np.inf    
#         for it in range(self.max_iter):
#             for j in range(self.f_users.shape[0]):
#                 u_w=np.diag(w[j].toarray())
#                 print(u_w.shape)
#                 a=np.dot(self.f_items.T, np.dot(u_w,self.f_items))+self.alpha*np.eye(self.factors)
#                 b=np.dot(self.f_items.T, np.dot(u_w,r[j].toarray().T))
#                 self.f_users[j] = np.linalg.solve(a,b).T
            
            
#             for j in range(self.f_items.shape[0]):
#                 i_w=np.diag(w.T[j].toarray())
                    
#                 a=np.dot(self.f_users.T, np.dot(i_w,self.f_users))+self.alpha*np.eye(self.factors)
#                 b=np.dot(self.f_users.T, np.dot(i_w,r.T[j].toarray()))
#                 self.f_items[j] = np.linalg.solve(a,b).T
            
#             u = np.sum(np.sum(self.f_users**2))
#             m = np.sum(np.sum(self.f_items**2))
#             cost = self.alpha*(u + m)
            
#             for i in range(self.f_users.shape[0]):
#                 r_pred=np.dot(self.f_users[i],self.f_items.T)
#                 cost += np.sum(np.sum(np.multiply(w[i],r[i].toarray()-r_pred)**2))
                    
#             print('iter %d, cost %.2f'%(it+1,cost))

#             if np.isnan(cost) or (last_cost > cost and last_cost-cost < self.epsilon) or last_cost<cost:
#                 break

#             last_cost = cost
                
              
#         return self

#     def predict(self,y_tst,items,uid_2_ix):
#         y_pred=[]

#         users=sorted(y_tst.pid.unique())
#         for i,u in enumerate(users):
#             f_user=self.f_users[uid_2_ix[u]]
#             r_pred=sorted([(i,r) for i,r in enumerate(np.dot(f_user,self.f_items.T))],key=lambda x:x[1],reverse=True)
            
#             seed_items=set(y_tst[y_tst.pid==u].track_id)
#             pred_items=[]
#             k=0
#             for j,r in enumerate(r_pred):
#                 itm=items[r[0]] 
#                 if itm not in seed_items:
#                     pred_items.append(itm)
#                     k+=1
#                     if k == self.rc:
#                         break
#             assert k==self.rc
#             y_pred.append(pred_items)
                    
#         return y_pred
            
# beta=10
# weights=csr_matrix((d*beta,(r,c)),shape=(len(playlists),len(tracks)),)
# print(weights[0].toarray().shape)
# wals=WAls(alpha=1e-1,beta=beta,factors=10,max_iter=100,epsilon=1e2)
# wals.fit(plays,weights)
# y_prd=wals.predict(playlist_tracks_tst,tracks,pid_2_ix)
# print('r_score %.3f'%r_prec(y_true[10:],y_prd))